In [1]:
# Cargamos las librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df= pd.read_csv('brandbyticket.csv')
del df['COUNT']
df.head()

,TransId,Marca
0,1100111835420180116170344,ABERCROMBIE & FITCH
1,1100111844820180117160026,ABERCROMBIE & FITCH
2,1100112130820180302152257,ABERCROMBIE & FITCH
3,1100112160420180308133218,ABERCROMBIE & FITCH
4,1100214403520180121174729,ABERCROMBIE & FITCH


In [2]:
group = df.groupby(["TransId","Marca"])["Marca"].count()
group.head(30)

TransId                   Marca                
100110732220170807130846  CAROLINA HERRERA         1
10025291520181031171328   LOEWE                    1
100510628620170927104604  LANCOME                  1
100515225320200814141635  CHANEL                   1
                          GUERLAIN                 1
                          JOHN VARVATOS            1
100515247520200921141632  GIVENCHY                 1
100515247820200922131533  CAROLINA HERRERA         1
100515251020200928143352  CHANEL                   1
                          JOHN VARVATOS            1
100515251420200929122740  CHANEL                   1
100515253220201001135817  DONNA KARAN              1
                          TOMMY HILFIGER           1
10059432620170306135152   CALVIN KLEIN             1
                          MOSCHINO                 1
1006966120170913121750    HUGO BOSS (VESTUARIO)    1
10081649520200130175204   CHRISTIAN DIOR           1
                          DOLCE & GABBANA          

## Transformación

Transformando las listas dentro de otras listas de las columnas

In [3]:
g = df.groupby('TransId').cumcount()
mux = pd.MultiIndex.from_product([df['TransId'].unique(), g.unique()])
transactions = (df.set_index(['TransId',g])
       .reindex(mux, fill_value='nan')
       .groupby(level=0)['Marca']
       .apply(lambda x: x.values.tolist())
       .tolist()
)
transactions

##print(transactions[1])

[['CAROLINA HERRERA',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['LOEWE',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['LANCOME',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['CHANEL',
  'JOHN VARVATOS',
  'GUERLAIN',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['GIVENCHY',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',


In [4]:
final_data = []
i=0
for transaction in transactions:
    cleanedList = [x for x in transactions[i] if str(x) != 'nan']
    final_data.append(cleanedList)
    i += 1
    
    
##final_data

## Obtención de reglas de asociación

Ahora se pude utilizar la función `apriori` para obtener los productos con un mínimo de soporte y todas las características que necesitamos en los hiperparámetros.


In [5]:
from apyori import apriori

rules = apriori(final_data, min_support = 0.003, min_confidence = 0.2, min_lift = 1, min_length = 2)  

In [6]:
results = list(rules)

print(results)


[RelationRecord(items=frozenset({'AGATHA RUIZ  DE LA PRADA', 'ANTONIO BANDERAS'}), support=0.0064971868014631175, ordered_statistics=[OrderedStatistic(items_base=frozenset({'AGATHA RUIZ  DE LA PRADA'}), items_add=frozenset({'ANTONIO BANDERAS'}), confidence=0.22008012691242507, lift=1.797452540154604)]), RelationRecord(items=frozenset({'SHAKIRA', 'ANTONIO BANDERAS'}), support=0.009149245580166632, ordered_statistics=[OrderedStatistic(items_base=frozenset({'SHAKIRA'}), items_add=frozenset({'ANTONIO BANDERAS'}), confidence=0.2582683517074482, lift=2.1093458611227596)]), RelationRecord(items=frozenset({'PUPA', 'CLARINS'}), support=0.003325988112172323, ordered_statistics=[OrderedStatistic(items_base=frozenset({'CLARINS'}), items_add=frozenset({'PUPA'}), confidence=0.3550245721064227, lift=6.146518729470772)]), RelationRecord(items=frozenset({'PUPA', 'H2O +'}), support=0.0030640367811420443, ordered_statistics=[OrderedStatistic(items_base=frozenset({'H2O +'}), items_add=frozenset({'PUPA'}),

In [7]:
results[0:2]

[RelationRecord(items=frozenset({'AGATHA RUIZ  DE LA PRADA', 'ANTONIO BANDERAS'}), support=0.0064971868014631175, ordered_statistics=[OrderedStatistic(items_base=frozenset({'AGATHA RUIZ  DE LA PRADA'}), items_add=frozenset({'ANTONIO BANDERAS'}), confidence=0.22008012691242507, lift=1.797452540154604)]),
 RelationRecord(items=frozenset({'SHAKIRA', 'ANTONIO BANDERAS'}), support=0.009149245580166632, ordered_statistics=[OrderedStatistic(items_base=frozenset({'SHAKIRA'}), items_add=frozenset({'ANTONIO BANDERAS'}), confidence=0.2582683517074482, lift=2.1093458611227596)])]

In [8]:
def inspect(results):
    rh          = [tuple(result[2][0][0]) for result in results]
    lh          = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(rh, lh, supports, confidences, lifts))

In [9]:
# Este comamdo crea un frame para ver los datos resultados
resultDataFrame=pd.DataFrame(inspect(results),
                columns=['X','Y','support','confidence','lift'])

In [11]:
resultDataFrame.sort_values(by='lift',ascending= False)

,X,Y,support,confidence,lift
2,"(CLARINS,)","(PUPA,)",0.003326,0.355025,6.146519
3,"(H2O +,)","(PUPA,)",0.003064,0.297931,5.158069
1,"(SHAKIRA,)","(ANTONIO BANDERAS,)",0.009149,0.258268,2.109346
0,"(AGATHA RUIZ DE LA PRADA,)","(ANTONIO BANDERAS,)",0.006497,0.220080,1.797453
